## Download and extract the dataset
---
### Download link expires on October 24, 2019

In [1]:
!rm -r *
!wget https://transfer.sh/fcpg1/dataset.zip
!unzip -q dataset.zip
!ls -l --block-size=M

--2019-12-05 17:19:21--  https://transfer.sh/fcpg1/dataset.zip
Resolving transfer.sh (transfer.sh)... 144.76.136.153
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 454498692 (433M) [application/x-zip-compressed]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 433.44M  7.78MB/s    in 53s     

2019-12-05 17:20:22 (8.21 MB/s) - ‘dataset.zip’ saved [454498692/454498692]

warning [dataset.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [dataset.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #363102).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 434M
drwxr-xr-x 5 root root   1M Dec  5 1

In [0]:
working_dir =   "/content"
train_dir   =   "/content/dataset/train/"
val_dir     =   "/content/dataset/val/"
test_dir    =   "/content/dataset/test/"

# আসল খেলা

In [3]:
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

Using TensorFlow backend.


# Training LeNet


In [0]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        color_mode='grayscale',
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        color_mode='grayscale',
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

Found 290135 images belonging to 122 classes.
Found 36223 images belonging to 122 classes.


In [0]:
filepath = working_dir + "/epoch:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('adam_0.01_training.log')

callbacks_list = [checkpoint, csv_logger]

model = Sequential()

model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))




# model.add(Conv2D(32, kernel_size = 5, activation='relu', input_shape = (28, 28, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, kernel_size = 4, activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, kernel_size = 3, activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))





model.add(Flatten())
# model.add(Dropout(0.4))
model.add(Dense(122, activation='softmax'))

# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
optimizer = optimizers.Adam(lr=0.01)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit_generator(
        train_generator,
        epochs=25,
        validation_data=validation_generator,
        callbacks=callbacks_list,
        verbose = 1)

Epoch 1/25
18134/18134 [==============================] - 260s 14ms/step - loss: 1.4541 - acc: 0.6297 - val_loss: 0.6855 - val_acc: 0.8124

Epoch 00001: val_acc improved from -inf to 0.81241, saving model to /content/epoch:001-val_acc:0.812.hdf5
Epoch 2/25
 5843/18134 [========>.....................] - ETA: 2:47 - loss: 0.7310 - acc: 0.8012

In [0]:
from google.colab import files
files.download('training.log') 